In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [1]:
import os 
import dotenv 

dotenv.load_dotenv()

True

In [2]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint

# GPT_API_KEY = os.environ.get("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [3]:
folder_path = '../data'

In [ ]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

In [5]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [3]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [6]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [11]:
query = ('결제 후 주문 취소가 가능한가요?')

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.498854] : 8
Question: Q[결제수단] 이미 주문했는데 결제 수단을 변경할 수 있나요?
Answer: 아니요. 상품을 주문/결제한 후에는 결제 수단 변경이 불가합니다. 상품이 아직 배송 전이라면 주문목록에서 주문을 취소한 후 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다. 상품이 이미 배송을 시작했다면 배송 완료 후 반품을 접수해 주시기 바랍니다. 단 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.
 주문 취소하기
마이쿠팡 → 주문목록 → 상품선택 → [주문취소] 
이후 각 단계에 해당하는 항목을 선택하여 취소를 완료합니다.
 쿠페이 결제수단 관리하기
마이쿠팡 → 결제수단∙쿠페이 → [결제수단 관리] 
이후 원하는 결제수단을 등록 및 삭제합니다.
 단순 변심 반품 비용 (클릭)
keywords: ORDER
count: 0 [{'row': 8.0, 'source': '../data/ORDER_main_qna_list.csv'}]
* [SIM=0.498439] : 9
Question: Q[취소 철회] 주문취소 신청을 철회(취소)하고 싶어요.
Answer: 배송 상품은 주문을 취소하면 바로 결제 취소가 적용되어 이후 주문취소를 철회(취소) 할 수 없습니다. 이미 주문취소를 하셨다면 재구매를 부탁드립니다.
keywords: CANCEL
count: 0 [{'row': 9.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
* [SIM=0.484393] : 44
Question: Q[취소] 신용카드로 결제한 주문을 취소했는데 카드대금이 청구되었습니다.
Answer: 신용카드로 결제한 주문을 취소하면 신용카드 승인 취소도 자동으로 진행됩니다. 그런데 만약 취소한 주문에 대해 카드대금 청구를 받으셨다면 이는 주문이 취소되기 전에 카드사에서 이미 청구서를 작성했기 때문입니다. 이 경우 청구된 카드대금이 취소되기까지는 특정 시간이 소요되며 이는 각 카드사마다 다릅니다. 

# LLM 질의 테스트

In [ ]:
%pip install langchain_google_genai

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # 사용할 모델을 지정합니다.
)

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.3보다 낮게 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
%pip install langchain_ollama

In [12]:
print("query: ", query)
print("=" * 80)
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')

query:  결제 후 주문 취소가 가능한가요?
Answer : 네, 고객님.
 결제 후 주문 취소 가능 여부는 상품 배송 여부에 따라 달라집니다.
 배송 전이라면 주문 취소가 가능하지만, 배송이 시작된 경우에는 취소가 어려울 수 있습니다.
 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.
 주문번호 확인 후, 취소 가능 여부와 절차를 안내해 드리겠습니다.



**조건별 대응 방법**

* **배송 전:** 고객님의 주문번호를 확인하여 주문 취소를 진행해 드립니다.
 취소 완료 후 결제 취소까지는 카드사별로 3~7일(주말 및 공휴일 제외) 정도 소요될 수 있다는 점 양해 부탁드립니다.


* **배송 중 또는 배송 완료:** 상품 배송이 시작되었거나 완료된 경우, 주문 취소는 불가능합니다.
 단순 변심으로 인한 반품을 원하시면 배송 완료 후 반품 접수를 도와드리겠습니다.
 이 경우 왕복 배송비가 부과될 수 있다는 점 미리 안내드립니다.


* **주문 취소 철회 요청:** 이미 주문 취소를 진행하셨다면, 취소를 철회할 수 없습니다.
 다시 구매를 원하시면 새롭게 주문해주시면 됩니다.


* **결제 수단 변경 요청:** 주문/결제 후에는 결제 수단 변경이 불가능합니다.
 배송 전이라면 주문 취소 후 원하는 결제 수단으로 재주문하셔야 합니다.



.


#### Ollama 버전 

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatOllama(
    model="llama3.1:8b",  # 사용할 모델을 지정합니다.
)

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
print("query: ", query)
print("=" * 80)
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')

query:  결제 후 주문 취소가 가능한가요?
Answer : 대본: 

"결제 후 주문 취소가 가능한가요? 불편을 드려 죄송합니다.
 주문 상태를 확인한 결과, 아직 배송 전이라면订单 취소 후 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다.
 상품이 이미 배송을 시작했다면 배송 완료 후 반품을 접수해 주시기 바랍니다.
 단, 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.
"

조건별 대응:

1.
 **주문 상태에 따라 주문 취소 가능 여부 확인**: 
  *  아직 배송 전이라면 : 
    +  "결제 후 주문 취소는 가능한가요?"
    +  "불편을 드려 죄송합니다.
 주문 상태를 확인한 결과, 아직 배송 전입니다.
"
    +  "이미 주문을 취소하실 수 있습니다.
 주문목록에서 주문을 취소하고 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다.
"
  *  이미배송을 시작했다면 : 
    +  "주문 상태를 확인한 결과, 이미 배송을 시작했습니다.
"
    +  "배송 완료 후 반품을 접수하실 수 있습니다.
 단, 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.
".
